In [1]:
%load_ext autoreload
%autoreload 2

In [29]:
import yaml

import astropy.io.fits as fits
import batoid
import galsim
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np

import batoid_rubin

%matplotlib widget

In [30]:
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    import matplotlib.pyplot as plt
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

In [44]:
def ptt(arr, eps=0.0):
    x = np.linspace(0, 2, arr.shape[0])
    y = np.linspace(0, 2, arr.shape[1])
    x, y = np.meshgrid(x, y)
    x -= np.mean(x)
    y -= np.mean(y)
    w = ~arr.mask
    
    zbasis = galsim.zernike.zernikeBasis(
        3, x[w], y[w], R_inner=eps, R_outer=1.0
    )
    coefs, *_ = np.linalg.lstsq(zbasis.T, arr[w], rcond=None)
    arr = arr - galsim.zernike.Zernike(
        coefs[:4], R_inner=eps, R_outer=1.0
    )(x, y)
    return arr

In [38]:
# Read field XY
with open("fieldXY.yaml") as f:
    data = yaml.safe_load(f)
field_x = np.array(data['x'])
field_y = np.array(data['y'])

In [39]:
fiducial = batoid.Optic.fromYaml("LSST_g_500.yaml")
wavelength = 500e-9

In [45]:
@ipywidgets.interact(
    imode=ipywidgets.BoundedIntText(value=10, min=0, max=49),
    ifield=ipywidgets.BoundedIntText(value=0, min=0, max=34),
    doptt=ipywidgets.Checkbox()
)
def f(imode, ifield, doptt):
    ts_phosim_opd = fits.getdata(f"phosim/opd_aos_dof_mode_{imode}_field_{ifield}.fits.gz")
    ts_phosim_opd0 = fits.getdata(f"../nominal/phosim/opd_nominal_field_{ifield}.fits.gz")
    ts_phosim_opd -= ts_phosim_opd0
    ts_phosim_opd = np.ma.masked_array(
        ts_phosim_opd, mask=ts_phosim_opd==0.0
    )
    
    # Perturb telescope
    builder = batoid_rubin.LSSTBuilder(fiducial, bend_dir="/Users/josh/src/batoid_rubin/scripts/bend_legacy/", fea_dir="/Users/josh/src/batoid_rubin/scripts/fea/")
    builder.dof[imode] = 1.0
    telescope = builder.build()
    
    # Convert from batoid -> phosim.
    # Implies flipping input theta_x and fliplr the output image
    batoid_opd = batoid.wavefront(
        telescope,
        -np.deg2rad(field_x[ifield]),
        np.deg2rad(field_y[ifield]),
        wavelength, nx=255, 
    ).array
    batoid_opd0 = batoid.wavefront(
        fiducial,
        -np.deg2rad(field_x[ifield]),
        np.deg2rad(field_y[ifield]),
        wavelength, nx=255, 
    ).array
    mask = batoid_opd.mask
    batoid_opd = np.fliplr(batoid_opd - batoid_opd0)
    # batoid in waves => microns
    batoid_opd *= wavelength*1e6
    
    if doptt:
        batoid_opd = ptt(batoid_opd, 0.61)
        ts_phosim_opd = ptt(ts_phosim_opd, 0.61)
    
    vmax = np.quantile(np.abs(batoid_opd[~mask]), 0.99)    
    fig, axes = plt.subplots(ncols=3, figsize=(8, 3))
    colorbar(axes[0].imshow(ts_phosim_opd, vmin=-vmax, vmax=vmax, cmap='seismic'))
    axes[0].set_title("ts_phosim")

    colorbar(axes[1].imshow(batoid_opd, vmin=-vmax, vmax=vmax, cmap='seismic'))
    axes[1].set_title("batoid")
    
    colorbar(axes[2].imshow(batoid_opd - ts_phosim_opd, vmin=-0.01*vmax, vmax=0.01*vmax, cmap='seismic'))
    axes[2].set_title("b - ph")

    for ax in axes:
        ax.set_aspect('equal')
    fig.tight_layout()
    plt.show()

interactive(children=(BoundedIntText(value=10, description='imode', max=49), BoundedIntText(value=0, descripti…